In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay

In [ ]:
client = EfdClient('usdf_efd')

In [ ]:

closes = []
winds = []
temps = []
startDay = 20230704
endDay = 20240206
dayObs = startDay
while dayObs <= endDay:
    nextDayObs = calcNextDay(dayObs)
    dayString = dayObsIntToString(dayObs)
    nextDayString = dayObsIntToString(nextDayObs)
    start = Time(f"{dayString}T16:00:00", scale='utc')
    end = Time(f"{nextDayString}T10:00:00", scale='utc')
    try:
        state = await client.select_time_series('lsst.sal.ATDome.logevent_mainDoorState', \
                                            ['state'],  start, end)
        wind = await client.select_time_series('lsst.sal.ESS.airFlow', \
                                                    ['speed'],  start, end, index=301)
        temp = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                    ['temperature0'],  start, end, index=301)
        if len(temp) == 0:
            temp = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                    ['temperatureItem0'],  start, end, index=301)
        close = state[state['state'] == 5]
        windMed = np.median(wind.values)
        tempMed = np.median(temp.values)
        print(len(state), len(wind), len(close), windMed, tempMed)
        if len(close) > 1:
            closes.append(len(close))
            winds.append(windMed)
            temps.append(tempMed)
        dayObs = calcNextDay(dayObs)
    except:
        dayObs = calcNextDay(dayObs)
        continue


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.set_title(f"Phantom Closes vs Wind Speed", fontsize=16)
ax.scatter(winds, closes)
ax.set_ylabel("Number of close events per night")
ax.set_xlabel("Median wind speed (m/s)")
ax.set_xlim(0,15)
plt.savefig("/home/c/cslage/u/AuxTel/dome/Shutter_Closes_vs_Wind_Speed_07Feb24.png")


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.set_title(f"Phantom Closes vs Temperature", fontsize=16)
ax.scatter(temps, closes)
ax.set_ylabel("Number of close events per night")
ax.set_xlabel("Median temperature (C)")
#ax.set_xlim(0,15)
plt.savefig("/home/c/cslage/u/AuxTel/dome/Shutter_Closes_vs_Temperature_07Feb24.png")


In [ ]:
%matplotlib inline
abs_temps = np.array(temps) + 273.15
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.set_title(f"Phantom Closes vs Temperature", fontsize=16)
ax.scatter(abs_temps, closes)
ax.set_yscale('log')
ax.set_ylabel("Number of close events per night")
ax.set_xlabel("Median temperature (K)")
#ax.set_xlim(0,15)
plt.savefig("/home/c/cslage/u/AuxTel/dome/Shutter_Closes_vs_Temperature_Activation_Eneergy_07Feb24.png")
